In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("philadelphia_reviews.csv")
df

,user_id,business_id,stars
0,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5
1,kSMOJwJXuEUqzfmuFncK4A,kxX2SOes4o-D3ZQBkiMRfA,2
2,mqBWACmaHflW4eh_Ofp16Q,kxX2SOes4o-D3ZQBkiMRfA,5
3,Z-xgVb4nM42943m2wbBkFw,kxX2SOes4o-D3ZQBkiMRfA,5
4,2SEoXb6r6hPKrl9V9VzBgA,kxX2SOes4o-D3ZQBkiMRfA,5
...,...,...,...
738683,reFwg-F-MCoRS1pA2YexMg,BEuGzy5gxtGyOEk8iwDfTg,5
738684,dh3UhfqpFSoCitl8WCrdfw,BEuGzy5gxtGyOEk8iwDfTg,5
738685,nXwPLYvazD_Nd2Sar3eSWA,BEuGzy5gxtGyOEk8iwDfTg,5
738686,MlOJCxiiB1TeZN-is2Q4SA,BEuGzy5gxtGyOEk8iwDfTg,5


In [10]:
class MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R = np.array(ratings)
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose
        
    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors ** 2))
        
    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i, j]) for i, j in zip(rows, columns)]

        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse()
            training_process.append((i+1, rmse))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.4f " % (i+1, rmse))
        return training_process

    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i,:].dot(self.Q[j,:].T)
        return prediction

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i,:] += self.alpha * (e * self.Q[j,:] - self.beta * self.P[i,:])
            self.Q[j,:] += self.alpha * (e * self.P[i,:] - self.beta * self.Q[j,:])


In [3]:
R_temp = df.pivot_table(index="user_id", columns="business_id", values="stars", aggfunc='mean').fillna(0)

In [4]:
R_temp[:10][:10]

business_id,-0M0b-XhtFagyLmsBtOe8w,-0PN_KFPtbnLQZEeb23XiA,-0TffRSXXIlBYVbb5AwfTg,-0eUa8TsXFFy0FCxHYmrjg,-0fvhILrC9UsQ6gLNpZlTQ,-1B9pP_CrRBJYPICE5WbRA,-1b2kNOowsPrPpBOK4lNkQ,-1yftDfgOUlxe_IACM5PFg,-2-ih3mE8KPyeKVIzpBfPQ,-32hOCwsuKd04WO-HQyYFg,...,zwd4dyQ5ovnjVojWfAuhMw,zwgBxmrmPBF6N2uRTUYXrQ,zxRmQ_FWVowh8rlzLCSURQ,zxY4DgtXsVHihSUpsmwamg,zxYD-C8GDnwuWDYxfD0spg,zy7uNOvpykrq-XlmDY_wHA,zyMkbavgHASQtqVwaock9A,zz-fcqurtm77bZ_rVvo2Lw,zz3E7kmJI2r2JseE6LAnrw,zzyx5x0Z7xXWWvWnZFuxlQ
user_id,,,,,,,,,,,,,,,,,,,,,
---r61b7EpVPkb4UVme5tA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--0kuuLmuYBe3Rmu0Iycww,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--13zE3NaRvLSrmfTVnFJA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--2tyArRmSoyKx5r-FVG0A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--2vR0DIsmQ6WfcSzKWigw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--4AjktZiHowEIBCMd4CZA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--4_p6Z3tKadJcr9Non_Vw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--6GckBYtTa4hj8pT09oAg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--6PFZka7og6Khaw6oyjvQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
mf = MF(R_temp, K = 30, alpha=0.001, beta=0.02, iterations=100, verbose=True)
train_process = mf.train()

Iteration: 10 ; Train RMSE = 1.1798 
Iteration: 20 ; Train RMSE = 1.1502 
Iteration: 30 ; Train RMSE = 1.1308 
Iteration: 40 ; Train RMSE = 1.1153 
Iteration: 50 ; Train RMSE = 1.1016 
Iteration: 60 ; Train RMSE = 1.0884 
Iteration: 70 ; Train RMSE = 1.0746 
Iteration: 80 ; Train RMSE = 1.0594 
Iteration: 90 ; Train RMSE = 1.0424 
Iteration: 100 ; Train RMSE = 1.0231 
